# Web Scraping
Note: All info given is from BeautifulSoup Lab 5.2

In [ ]:
# Step 0: Imports
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

# Step 1: Creating a soup opject from the website homepage
res = requests.get('http://chubbygrub.com')
print(res)
soup = bs(res.content, 'lxml')
soup

# Step 2: Scraping homepage for each restaurant name
restaurants_section = soup.find('div', {'class':'restaurant-buttons'})
print(restaurants_section)
#def function():                         #Make a scraping function of some kind in future?
restaurants = []
for restaurant_link in restaurants_section.find_all('a',{'class': 'btn btn-lg btn-primary'}):
    restaurant = {}                                             # Initiates dictionary
    restaurant['name'] = restaurant_link.text                   # Adds the name of the restaurant
    restaurant['slug'] = restaurant_link['href'].split('/')[-1] # Adds slug(?) from href(?)
    restaurants.append(restaurant)
print(restaurants)
print(restaurants_section.find_all('a',{'class': 'btn btn-lg btn-primary'}))

# Step 3: Using slug to scrape each restaurant's page and creating single list of food dicts
foods = []                   # Init empty list
for restaurant in restaurants: # for loop for each name and slug
    print('Scraping: ', restaurant['name']) # Print current restaurant
    # Get request for each restaurant with dis list sitch
    restaurant_res = requests.get('http://chubbygrub.com/restaurants/{}'.format(restaurant['slug'])) 
    restaurant_soup = bs(restaurant_res.content, 'lxml') # Init Soup
    table = restaurant_soup.find('table', {'id': 'items'}) # NAb the food tables
    for row in table.find('tbody').find_all('tr'):  #Looping through each row
        cells = row.find_all('td') # Creating variable for <td/>
        food = {} # Init Dict
        food['restaurant'] = restaurant['name']    # These are all adding the thing...
        food['name']       = cells[0].text         # that you'd think they're adding
        food['category']   = cells[1].text.strip() # .strip() is removing white space
        food['calories']   = cells[2].text
        food['fat']        = cells[3].text
        food['carbs']      = cells[4].text
        foods.append(food)

# Step 4: Creating a pandas from list of foods
df = pd.DataFrame(foods)
print(df.shape)
print(df.head())

# Step 5: Exporting to CSV
df.to_csv('./fast_foods_date', index=False)

## Ridge Lasso and Elastinet code from Kobe Shots

In [ ]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

y = df.SHOTS_MADE
X = df.iloc[:,1:]
ss = StandardScaler()
Xs = ss.fit_transform(X)

ridge = RidgeCV(np.logspace(0, 5, 200), cv=10)
ridge.fit(Xs, y)
print(ridge.alpha_)

ridge_scores = cross_val_score(Ridge(alpha = ridge.alpha_),
                               Xs, y, cv=10)
print(np.mean(ridge_scores))
# This is much better because we've optomized with Ridge.
#(Notes: Multicolinearity.  Ridge manages multicolinearity)

lasso = LassoCV(n_alphas = 500, cv = 10, verbose = 0)
lasso.fit(Xs, y)
print(lasso.alpha_)

lasso_scores = cross_val_score(Lasso(alpha=lasso.alpha_),
                               Xs, y, cv = 10)
print(np.mean(lasso_scores))
# It's marginally better than Ridge.
# Note: Lasso focuses on feature selection.

lasso_model = Lasso(alpha=lasso.alpha_).fit(Xs, y)
lasso_coefs = pd.DataFrame({'variable': X.columns,
                            'coefs': lasso.coef_,
                            'abs_coef': np.abs(lasso.coef_)})
lasso_coefs.sort_values('abs_coef', 
                        inplace = True, 
                        ascending = False)
lasso_coefs.head()

enet = ElasticNetCV(l1_ratio=np.linspace(0.01, 1.0, 25),
                    n_alphas = 100, cv = 10)
enet.fit(Xs, y)
print(enet.alpha_)
print(enet.l1_ratio_)

enet_scores = cross_val_score(ElasticNet(alpha=enet.alpha_,
                                        l1_ratio=enet.l1_ratio_),
                              Xs, y, cv = 10)
print(np.mean(enet_scores))
# It's doing about the same as the Lasso.

#Some code for running through Cross Val Scores between two numbers of folds
for i in range (5,11):
    lin_reg = LinearRegression()
    print('\n Folds:', i)
    #print('Cross Val Scores:',cross_val_score(lin_reg, X, y, cv=i))
    print('Cross Val Pred R2:', 
          metrics.r2_score(y, cross_val_predict(lin_reg, X, y, cv=i)))
    

In [ ]:
## Gridsearch
cvec = CountVectorizer(analyzer = "word",
                       tokenizer = tokenizer.tokenize,
                       preprocessor = None,
                       stop_words = 'english',
                       min_df = 2, # This can be a float representing it's proportion of the documents
                       max_df = None
                       ngram_range = (min,max),
                      max_features = None) 

In [ ]:
hyperparameters = {
    ''
}
from sklearn.model_selection import GridSearchCV

GridSearchCV()

from sklearn.pipeline import Pipeline, make_pipeline

Pipeline()
make_pipeline()